# Import

In [1]:
%matplotlib widget

In [2]:
import os

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# Define

In [3]:
def my_plot(table, plot_args, title, xlabel='iteration', ylabel='accuracy', ylim=[0, 1]):
    fig, ax = plt.subplots()
    lo = ax.plot(
        table.columns.values,
        table.transpose(),
        **plot_args
    )
    ax.legend(iter(lo), table.index.values, loc='best')

    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_ylim(*ylim)

    plt.tight_layout()
    return fig
    

In [4]:
def display_val_summary(modifier, iteration, plot_args, eval_dir, ylim=[0,1], save_plot=False, plot_type='jpg'):
    fname = os.path.join(eval_dir, f'r{iteration}', 'tables', f'configs.{modifier}.csv')
    summary_table = pd.read_csv(fname, index_col = 0)
    print(summary_table)
    fig = my_plot(summary_table, plot_args, modifier, ylim=ylim)
    
    if save_plot:
        assert not plot_type is None
        fig.savefig(os.path.join(eval_dir, f'r{iteration}', 'plots', f'configs.{modifier}.{plot_type}'))


def display_itereval_summary(sub_keys, iteration, plot_args, eval_dir, ylim=[0,1], save_plot=False, plot_type='jpg'):
    rep = {
        '/': '-',
        ';': '--',
    }
    
    fname_key = '.'.join(sub_keys.values())
    for old_char, new_char in rep.items():
        fname_key = fname_key.replace(old_char, new_char)
    fname = os.path.join(eval_dir, f'r{iteration}', 'tables', f'iterevals.{fname_key}.csv')
    summary_table = pd.read_csv(fname, index_col = 0)
    print(summary_table)
    title = fname_key = '-'.join(sub_keys.values())
    fig = my_plot(summary_table, plot_args, title, ylim=ylim)
    
    if save_plot:
        assert not plot_type is None
        fig.savefig(os.path.join(eval_dir, f'r{iteration}', 'plots', f'iterevals.{fname_key}.{plot_type}'))
    

# Summarize

In [5]:
repo = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))

## Validation Sets

In [6]:
mod = 'standard'  # either standard or hyp
iteration = 1
eval_dir = os.path.join(repo, 'eval_summary')
save_plot = False
plot_type = 'jpg'

ylim = [0,1]
plot_args = {
    'linestyle': '-',
    'marker': 'o',
}

display_val_summary(mod, iteration, plot_args, eval_dir, ylim=ylim, save_plot=save_plot, plot_type=plot_type)

                 1
baseline  0.843813
LotS      0.823529
LitL      0.850932


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Iterevals

In [7]:
glue_case_keys = pd.read_csv('glue_case_keys.csv', header=0, index_col=False)
hans_case_keys = pd.read_csv('hans_case_keys.csv', header=0, index_col=False)

In [8]:
'''
Cases for GLUE:
    combined, Knowledge, Lexical Semantics, Logic, Predicate-Argument Structure

Cases for HANS:
    combined, constituent, lexical_overlap, subsequence

'''

glue_case = 'Knowledge'
print(glue_case_keys.loc[glue_case_keys['case'] == glue_case, 'subcase'])
print('='*90)

hans_case = 'constituent'
print(hans_case_keys.loc[hans_case_keys['case'] == hans_case, 'subcase'])

1           combined
2       Common sense
3    World knowledge
Name: subcase, dtype: object
1                   ce_adverb
2       ce_after_since_clause
3              ce_conjunction
4     ce_embedded_under_since
5      ce_embedded_under_verb
6                   cn_adverb
7          cn_after_if_clause
8              cn_disjunction
9        cn_embedded_under_if
10     cn_embedded_under_verb
11                   combined
Name: subcase, dtype: object


In [9]:
sub_keys = {
    'dataset': 'hans',     # either hans or glue
    'case': 'combined',    # combined or specific to respective itereval set
    'subcase': 'combined', # combined or specific to respective itereval set
    'label': 'combined',   # combined or [entailment, neutral, contradiction] for glue, [entailment, non-entailment] for hans
}
iteration = 1
eval_dir = os.path.join(repo, 'eval_summary')
save_plot = False
plot_type = 'jpg'

ylim = [0,1]
plot_args = {
    'linestyle': '-',
    'marker': 'o',
}

display_itereval_summary(sub_keys, iteration, plot_args, eval_dir, ylim=ylim, save_plot=save_plot, plot_type=plot_type)

                 1
baseline  0.269033
LotS      0.222700
LitL      0.253600


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …